In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

In [2]:
# model parameters
project = ['1', '2', '3', '4', '5']
cost = [100, 200, 150, 75, 300]
profit = [1, 1.8, 2, 1.5, 3.6]



In [3]:
# initialize model object
mymodel = glp.Solver('Kloos Industry', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)



In [4]:
# create decision variables
select = list(range(len(project)))
for j in range(len(project)):
    select[j] = mymodel.IntVar(0,1, project[j])
    
print('select = ', select)

select =  [1, 2, 3, 4, 5]


In [5]:
# create objective function
tot_npv = mymodel.Objective()
tot_npv.SetMaximization()
for j in range(len(project)):
    tot_npv.SetCoefficient(select[j], profit[j])

#display objective function
for j in range(len(project)):
    print(select[j], tot_npv.GetCoefficient(select[j]))

1 1.0
2 1.8
3 2.0
4 1.5
5 3.6


In [6]:
# create constraints
constr = list()
#i. Choose no fewer than three projects.
constr.append(mymodel.Constraint(3,5))
for i in range(len(project)):
    constr[0].SetCoefficient(select[i], 1)
    

In [7]:
#ii. If project 3 is chosen, project 4 must be chosen.
constr.append(mymodel.Constraint(0,0))
constr[1].SetCoefficient(select[2], -1)
constr[1].SetCoefficient(select[3], 1)

In [8]:
#iii. If project 1 is chosen, project 5 must not be chosen.
constr.append(mymodel.Constraint(0,1))
constr[2].SetCoefficient(select[0], 1)
constr[2].SetCoefficient(select[4], 1)

In [9]:
#iv. The budget is 450.
constr.append(mymodel.Constraint(0,450))
for i in range(len(project)):
    constr[3].SetCoefficient(select[i], cost[i])
    

In [10]:
#v. No more than two of projects 1, 2, and 3 can be chosen.
constr.append(mymodel.Constraint(0,2))
constr[4].SetCoefficient(select[0], 1)
constr[4].SetCoefficient(select[1], 1)
constr[4].SetCoefficient(select[2], 1)

In [11]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % tot_npv.Value())

# Display optimal solution
for j in range(len(project)):
    print('%s %4.2f' % (project[j], select[j].solution_value()))
    

Solution Status = 0
Number of variables = 5
Number of constraints = 5
Optimal Solution:
Optimal Value = 5.30
1 0.00
2 1.00
3 1.00
4 1.00
5 0.00


In [13]:
# store coefficients and RHS for cut constraint
a = list()
M = 0

for i in range(len(select)):
    if select[i].solution_value() == 1:
        a.append(1)
        M+=1
    elif select[i].solution_value() == 0:
        a.append(-1)
print(M, a)

3 [-1, 1, 1, 1, -1]


In [14]:
#add cut constraint
cut_constr = mymodel.Constraint(-mymodel.infinity(), M-1)
for i in range(len(project)):
    cut_constr.SetCoefficient(select[i], a[i])

    
#re-optimize and display next solution
mymodel.Solve()
print('Optimal Value =', tot_npv.Value())
i = 0
for i in range(len(project)):
    if select[i].solution_value() == 1:
            print('%s %4.2f' % (project[i], select[i].solution_value()))
        
# relax cut constraint
cut_constr.SetBounds(-mymodel.infinity(),mymodel.infinity())

Optimal Value = 4.5
1 1.00
3 1.00
4 1.00
